# Using the rs-server catalog

This tutorial is meant the basic usage of the rs-server catalog service.

## Introduction

The rs-server catalog service exposes a REST API 
on the /catalog endpoint of the rs-server.

Each user owns a dedicated STAC catalog.
It can be accessed at the /catalog/{user} endpoint.
Each catalog is following the STAC API.

Let's illustrate all this with some examples.

## Configuration

In [ ]:
# Set local or cluster configuration
import os

if os.getenv("RSPY_LOCAL_MODE") == "1":
    RS_SERVER_ROOT_URL = "http://rs-server-catalog:8000"
    HEADERS={}
    local_mode = True
else:
    RS_SERVER_ROOT_URL = os.environ["RSPY_WEBSITE"]
    HEADERS={"headers": {"x-api-key": os.environ["RSPY_APIKEY"]}}
    local_mode = False

print(f"Using: {RS_SERVER_ROOT_URL}")

import requests
import json

In [ ]:
# Use boto3 for S3 operations

TEMP_BUCKET = "rs-cluster-temp"
FINAL_BUCKET = "rs-cluster-catalog"

!pip install boto3
import boto3
import os
s3_session = boto3.session.Session()

s3_client = s3_session.client(
    service_name="s3",    
    # Note: the S3_ACCESSKEY, S3_SECRETKEY and S3_ENDPOINT are given 
    # in the docker-compose.yml or ~/.s3cfg file.
    aws_access_key_id=os.environ["S3_ACCESSKEY"],
    aws_secret_access_key=os.environ["S3_SECRETKEY"],
    endpoint_url=os.environ["S3_ENDPOINT"],
    region_name=os.environ["S3_REGION"],
)
    
# Create the buckets if they don't exist (only in local mode)
if local_mode:
    buckets = [TEMP_BUCKET, FINAL_BUCKET] # bucket names under S3_ENDPOINT
    for b in buckets:
        if b not in [bucket["Name"] for bucket in s3_client.list_buckets()["Buckets"]]:
            s3_client.create_bucket(Bucket=b)

In [ ]:
# Clean previous executions

for collection in ("esmeralda:S1_L1", "quasimodo:S1_L1"):
    requests.delete(f"{RS_SERVER_ROOT_URL}/catalog/collections/{collection}", **HEADERS)

for item_name in ("cadip/item1.dataset", "cadip/item2.dataset"):
    for bucket in (TEMP_BUCKET, FINAL_BUCKET):
        s3_client.delete_object(Bucket=bucket, Key=item_name)

In [ ]:
# Create empty dummy datasets for this demo
import tempfile
for item_name in ("cadip/item1.dataset", "cadip/item2.dataset"):
    with tempfile.NamedTemporaryFile() as tmp:
        s3_client.upload_file (tmp.name, TEMP_BUCKET, item_name)

# Save the current datetime
from datetime import datetime
demo_start_time = datetime.now().astimezone()

## Add the collection S1_L1 in the esmeralda catalog

In [ ]:
collection = {
            "id": "S1_L1",
            "type": "Collection",
            "description": "The S1_L1 collection for Esmeralda user.",
            "stac_version": "1.0.0",
            "owner": "esmeralda"
        }

post_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections", json=collection, **HEADERS)
post_response.raise_for_status()

response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/esmeralda:S1_L1", **HEADERS)
response.raise_for_status()

collection = json.loads(response.content)
collection

The previous commands create the S1_L1 collection in the esmeralda catalog.
Then, it gets the esmeralda collections again.
This time, there is the S1_L1 collection, previously added.

As we can see, the user doesn't have to add the owner_id to the collection_id. It is automatically added during the post.

## Add the collection S1_L1 in the quasimodo catalog

In [ ]:
collection = {
            "id": "S1_L1",
            "type": "Collection",
            "description": "The S1_L1 collection for Quasimodo user.",
            "stac_version": "1.0.0",
            "owner": "quasimodo"
        }

post_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections", json=collection, **HEADERS)
post_response.raise_for_status()

response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/quasimodo:S1_L1", **HEADERS)
response.raise_for_status()

collection = json.loads(response.content)
collection

Preview the content of catalog and temporary bucket, pushing a feature will automatically transfer files to catalog bucket, and update links.

## Add an item in the collection S1_L1 in the esmeralda catalog

In [ ]:
item_0 = {
            "id": "item_0",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-94.6334839, 37.0595608],
                        [-94.6334839, 37.0332547],
                        [-94.6005249, 37.0332547],
                        [-94.6005249, 37.0595608],
                        [-94.6334839, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 2500,
                "height": 2500,
                "datetime": "2000-02-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "esmeralda",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item1.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }

post_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/esmeralda:S1_L1/items", json=item_0, **HEADERS)
post_response.raise_for_status()

item_response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/esmeralda:S1_L1/items", **HEADERS)
item_response.raise_for_status()

item = json.loads(item_response.content)
item

Preview again the content of buckets, to make sure that assests were moved.

In [ ]:
# Check that the asset was copied to the final catalog bucket
item1_time = s3_client.head_object(Bucket=FINAL_BUCKET, Key="cadip/item1.dataset")["LastModified"]
assert item1_time > demo_start_time

The previous commands create the first feature in the esmeralda S1_L1 collection. Then, it gets the the feature created.

Let's look at its content.

## Add an item in the collection S1_L1 in the quasimodo catalog

In [ ]:
item_0 = {
            "id": "item_0",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-94.6334839, 37.0595608],
                        [-94.6334839, 37.0332547],
                        [-94.6005249, 37.0332547],
                        [-94.6005249, 37.0595608],
                        [-94.6334839, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 2500,
                "height": 2500,
                "datetime": "2000-02-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "quasimodo",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item1.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }

post_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/quasimodo:S1_L1/items", json=item_0, **HEADERS)
post_response.raise_for_status()

item_response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/quasimodo:S1_L1/items", **HEADERS)
item_response.raise_for_status()

item = json.loads(item_response.content)
item

## Get the first item from Esmeralda S1_L1 collection

In [ ]:
import requests
import json

get_response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/esmeralda:S1_L1/items/item_0", **HEADERS)
get_response.raise_for_status()

item = json.loads(get_response.content)
item

The previous commands will display a single item from esmeralda S1_L1 collection

## Add 2 new items in esmeralda S1_L1 collection and search all items with the datetime: 

In [ ]:
item_1 = {
            "id": "item_1",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-94.6334839, 37.0595608],
                        [-94.6334839, 37.0332547],
                        [-94.6005249, 37.0332547],
                        [-94.6005249, 37.0595608],
                        [-94.6334839, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 2500,
                "height": 2500,
                "datetime": "2000-03-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "esmeralda",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item1.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }

item_2 = {
            "id": "item_2",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-94.6334839, 37.0595608],
                        [-94.6334839, 37.0332547],
                        [-94.6005249, 37.0332547],
                        [-94.6005249, 37.0595608],
                        [-94.6334839, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 2500,
                "height": 2500,
                "datetime": "2000-03-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "esmeralda",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item2.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }

post = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/esmeralda:S1_L1/items", json=item_1, **HEADERS)
post.raise_for_status()
post = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/esmeralda:S1_L1/items", json=item_2, **HEADERS)
post.raise_for_status()

In [ ]:
parameters = {"collections": ["S1_L1"], "filter": "datetime='2000-03-02T00:00:00Z' AND owner_id='esmeralda'"}
search_response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/search", params=parameters, **HEADERS)
search_response.raise_for_status()

result = json.loads(search_response.content)
result

The previous command will display the result from the search endpoint with specifics parameters such as the collection name or the datetime.

In [ ]:
json_parameters = {
    "collections": ["S1_L1"],
    "filter-lang": "cql2-json",
    "filter": {
        "op": "and",
        "args": [
            {"op": "=", "args": [{"property": "owner_id"}, "esmeralda"]},
            {"op": "=", "args": [{"property": "datetime"}, "2000-03-02T00:00:00Z"]},
        ],
    },
}

search_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/search", json=json_parameters, **HEADERS)
search_response.raise_for_status()

result = json.loads(search_response.content)
result

We can do the same operation using a post request.

## Update the item_0 from the Esmeralda S1_L1 collection

In [ ]:
new_item_0 = {
            "id": "item_0",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-100, 37.0595608],
                        [-108, 37.0332547],
                        [-100, 37.0332547],
                        [-111, 37.0595608],
                        [-100, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 5000,
                "height": 2500,
                "datetime": "2014-02-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "esmeralda",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item2.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }

update_response = requests.put(f"{RS_SERVER_ROOT_URL}/catalog/collections/esmeralda:S1_L1/items/item_0", json=new_item_0, **HEADERS)
update_response.raise_for_status()

result = json.loads(update_response.content)
result

The previous command will update the item_0 from the Esmeralda S1_L1 collection by changing geometry, width and datetime.

In [ ]:
# Check that the asset was copied to the final catalog bucket
item1_time = s3_client.head_object(Bucket=FINAL_BUCKET, Key="cadip/item2.dataset")["LastModified"]
assert item1_time > demo_start_time

## Update the collection S1_L1 from the Esmeralda catalog

In [ ]:
new_collection_S1_L1 = {
            "id": "S1_L1",
            "type": "Collection",
            "description": "This is the new description for the S1_L1 collection for Esmeralda user.",
            "stac_version": "1.0.0",
            "owner": "esmeralda"
        }

update_response = requests.put(f"{RS_SERVER_ROOT_URL}/catalog/collections", json=new_collection_S1_L1, **HEADERS)
update_response.raise_for_status()

result = json.loads(update_response.content)
result

The previous command will update the collection S1_L1 from the Esmeralda catalog by changing the collection description.

## Delete an item from the Esmeralda S1_L1 collection

In [ ]:
delete_response = requests.delete(f"{RS_SERVER_ROOT_URL}/catalog/collections/esmeralda:S1_L1/items/item_2", **HEADERS)
delete_response.raise_for_status()

result = json.loads(delete_response.content)
result

## Delete the collection S1_L1 from the Esmeralda catalog

In [ ]:
delete_response = requests.delete(f"{RS_SERVER_ROOT_URL}/catalog/collections/esmeralda:S1_L1", **HEADERS)
delete_response.raise_for_status()

result = json.loads(delete_response.content)
result

## Delete the collection S1_L1 from the Quasimodo catalog

In [ ]:
delete_response = requests.delete(f"{RS_SERVER_ROOT_URL}/catalog/collections/quasimodo:S1_L1", **HEADERS)
delete_response.raise_for_status()

result = json.loads(delete_response.content)
result

## Clear files from catalog bucket

In [ ]:
for item_name in ("cadip/item1.dataset", "cadip/item2.dataset"):
    for bucket in (TEMP_BUCKET, FINAL_BUCKET):
        s3_client.delete_object(Bucket=bucket, Key=item_name)

## To go further


---
**NOTE**

STAC Swagger UI: http://localhost:8003/api.html

---

STAC Browser at http://localhost:8081

Search for esmeralda catalog at http://localhost:8003/catalog/esmeralda